### Imports

In [93]:
import tensorflow as tf
import numpy as np
import os
from datetime import datetime
import re
from shutil import copyfile, rmtree
import argparse
import math
import random
import glob
import pandas as pd
import io
import xml.etree.ElementTree as ET

import tensorflow.compat.v1 as tf1
from PIL import Image
from object_detection.utils import dataset_util, label_map_util
from collections import namedtuple

### Define constants

In [94]:
PATH = os.getcwd()
#choose PATH + "\data\stop_sign\OIDV4_toolkit_sign" to train on OIVD4 data, 
#choose PATH + "\data\stop_sign\kaggle_stop_sign" to train on kaggle data, 
#choose PATH + "\data\stop_sign\all_data" to train on all data
PATH_IMAGE = PATH + "\\images"
PATH_MODEL = "\\models"
print(PATH_IMAGE)

D:\roscore_projects\final_tp_robotics\robotics_train_sign_data\images


### Create train and test images folder (seems mandatory in the tutorial)

In [108]:
def iterate_dir(source=PATH_IMAGE, dest=PATH_IMAGE, ratio=0.1, copy_xml=True):
    source = source.replace('\\', '/')
    dest = dest.replace('\\', '/')
    train_dir = os.path.join(dest, 'train')
    test_dir = os.path.join(dest, 'test')
    reset = 0
    
    if not os.path.exists(train_dir):
        os.makedirs(train_dir)
    else:
        reset = 1
        
        
    if not os.path.exists(test_dir):
        os.makedirs(test_dir)
    else:
        reset = 1
        
    if reset == 1:
        r = glob.glob(PATH_IMAGE+"\\train")
        s = glob.glob(PATH_IMAGE+"\\test")
        try:
            rmtree(PATH_IMAGE+"\\train")
            os.makedirs(train_dir)
        except FileNotFoundError:
            print("File does not exist")
            print("file could not be removed but 0 bugs, ",e)
            
            
        try:
            rmtree(PATH_IMAGE+"\\test")
            os.makedirs(test_dir)
        except FileNotFoundError:
            print("File does not exist")
            print("file could not be removed but 0 bugs, ",e)
            

    images = [f for f in os.listdir(source)
              if re.search(r'([a-zA-Z0-9\s_\\.\-\(\):])+(.jpg|.jpeg|.png)$', f)]

    num_images = len(images)
    num_test_images = math.ceil(ratio*num_images)

    for i in range(num_test_images):
        idx = random.randint(0, len(images)-1)
        filename = images[idx]
        copyfile(os.path.join(source, filename),
            os.path.join(test_dir, filename))
        if copy_xml:
            xml_filename = os.path.splitext(filename)[0]+'.xml'
            copyfile(os.path.join(source, xml_filename),
                     os.path.join(test_dir,xml_filename))
        images.remove(images[idx])

    for filename in images:
        copyfile(os.path.join(source, filename),
                 os.path.join(train_dir, filename))
        if copy_xml:
            xml_filename = os.path.splitext(filename)[0]+'.xml'
            copyfile(os.path.join(source, xml_filename),
                     os.path.join(train_dir, xml_filename))
            
iterate_dir()

### Create TF Records for train and test data

In [109]:
#Several functions to transform the data inside train and test data

def xml_to_csv(path):
    """Iterates through all .xml files (generated by labelImg) in a given directory and combines
    them in a single Pandas dataframe.

    Parameters:
    ----------
    path : str
        The path containing the .xml files
    Returns
    -------
    Pandas DataFrame
        The produced dataframe
    """

    xml_list = []
    for xml_file in glob.glob(path + '/*.xml'):
        tree = ET.parse(xml_file)
        root = tree.getroot()
        for member in root.findall('object'):
            value = (root.find('filename').text,
                     int(root.find('size')[0].text),
                     int(root.find('size')[1].text),
                     member[0].text,
                     int(member[4][0].text),
                     int(member[4][1].text),
                     int(member[4][2].text),
                     int(member[4][3].text)
                     )
            xml_list.append(value)
    column_name = ['filename', 'width', 'height',
                   'class', 'xmin', 'ymin', 'xmax', 'ymax']
    xml_df = pd.DataFrame(xml_list, columns=column_name)
    return xml_df


def class_text_to_int(row_label, label_map_dict):
    return label_map_dict[row_label]

def split(df, group):
    data = namedtuple('data', ['filename', 'object'])
    gb = df.groupby(group)
    return [data(filename, gb.get_group(x)) for filename, x in zip(gb.groups.keys(), gb.groups)]

def create_tf1_example(group, path, label_map_dict):
    with tf1.gfile.GFile(os.path.join(path, '{}'.format(group.filename)), 'rb') as fid:
        encoded_jpg = fid.read()
    encoded_jpg_io = io.BytesIO(encoded_jpg)
    image = Image.open(encoded_jpg_io)
    width, height = image.size

    filename = group.filename.encode('utf8')
    image_format = b'jpg'
    xmins = []
    xmaxs = []
    ymins = []
    ymaxs = []
    classes_text = []
    classes = []

    for index, row in group.object.iterrows():
        xmins.append(row['xmin'] / width)
        xmaxs.append(row['xmax'] / width)
        ymins.append(row['ymin'] / height)
        ymaxs.append(row['ymax'] / height)
        classes_text.append(row['class'].encode('utf8'))
        classes.append(class_text_to_int(row['class'], label_map_dict))

    tf1_example = tf1.train.Example(features=tf1.train.Features(feature={
        'image/height': dataset_util.int64_feature(height),
        'image/width': dataset_util.int64_feature(width),
        'image/filename': dataset_util.bytes_feature(filename),
        'image/source_id': dataset_util.bytes_feature(filename),
        'image/encoded': dataset_util.bytes_feature(encoded_jpg),
        'image/format': dataset_util.bytes_feature(image_format),
        'image/object/bbox/xmin': dataset_util.float_list_feature(xmins),
        'image/object/bbox/xmax': dataset_util.float_list_feature(xmaxs),
        'image/object/bbox/ymin': dataset_util.float_list_feature(ymins),
        'image/object/bbox/ymax': dataset_util.float_list_feature(ymaxs),
        'image/object/class/text': dataset_util.bytes_list_feature(classes_text),
        'image/object/class/label': dataset_util.int64_list_feature(classes),
    }))
    return tf1_example


#Use functions to achieve the transformation
def create_TF_record(labels_path, output_path, image_dir, xml_dir, csv_path=None):
    label_map = label_map_util.load_labelmap(labels_path)
    label_map_dict = label_map_util.get_label_map_dict(label_map)

    writer = tf1.python_io.TFRecordWriter(output_path)
    path = os.path.join(image_dir)
    examples = xml_to_csv(xml_dir)
    grouped = split(examples, 'filename')
    for group in grouped:
        tf1_example = create_tf1_example(group, path, label_map_dict)
        writer.write(tf1_example.SerializeToString())
    writer.close()
    print('Successfully created the TFRecord file: {}'.format(output_path))
    if csv_path is not None:
        examples.to_csv(csv_path, index=None)
        print('Successfully created the CSV file: {}'.format(csv_path))
    return

#PATH+"\annotations\label_map.pbtxt"
create_TF_record(PATH+"\\annotations\\label_map.pbtxt".replace("\\", "/"), 
    PATH+"\\annotations\\train.record", PATH_IMAGE+"\\train", PATH_IMAGE+"\\train", None)
create_TF_record(PATH+"\\annotations\\label_map.pbtxt".replace("\\", "/"),
    PATH+"\\annotations\\test.record", PATH_IMAGE+"\\test", PATH_IMAGE+"\\test", None)

Successfully created the TFRecord file: D:\roscore_projects\final_tp_robotics\robotics_train_sign_data\annotations\train.record
Successfully created the TFRecord file: D:\roscore_projects\final_tp_robotics\robotics_train_sign_data\annotations\test.record


### Get images and normalize them

In [42]:
#print all filenames in directory 
for f in os.listdir(PATH_IMAGE):
    print(f)

001fba4728544c2e.jpg
001fba4728544c2e.xml
0026b117bb17880a.jpg
0026b117bb17880a.xml
0062d9a5fb4913ce.jpg
0062d9a5fb4913ce.xml
0121f9c615042091.jpg
0121f9c615042091.xml
01550f1b9680c4ce.jpg
01550f1b9680c4ce.xml
017f45b1410d3ddb.jpg
017f45b1410d3ddb.xml
01a021697ff040bd.jpg
01a021697ff040bd.xml
0257eeda008582ab.jpg
0257eeda008582ab.xml
02c05d5fcae7b5d9.jpg
02c05d5fcae7b5d9.xml
02c3ca715cba6581.jpg
02c3ca715cba6581.xml
036ca96be461e0e9.jpg
036ca96be461e0e9.xml
0378cdb594c70f0c.jpg
0378cdb594c70f0c.xml
03e7e1bdf3f1a786.jpg
03e7e1bdf3f1a786.xml
03faf8b9a0ba1fff.jpg
03faf8b9a0ba1fff.xml
04300b302e117981.jpg
04300b302e117981.xml
04405957b0778fbb.jpg
04405957b0778fbb.xml
049f652ae942fb61.jpg
049f652ae942fb61.xml
04afaf46b27357dd.jpg
04afaf46b27357dd.xml
0526a593b144a52b.jpg
0526a593b144a52b.xml
05d4a1a8d2d10d3d.jpg
05d4a1a8d2d10d3d.xml
0631bf1ccff448a2.jpg
0631bf1ccff448a2.xml
064c8f87036a9dfe.jpg
064c8f87036a9dfe.xml
06db4f83f1ef0286.jpg
06db4f83f1ef0286.xml
06e9a81ceb6769a3.jpg
06e9a81ceb676

64dcc5967189fa45.jpg
64dcc5967189fa45.xml
6576c94a51c3435f.jpg
6576c94a51c3435f.xml
65e66cd2b51b6031.jpg
65e66cd2b51b6031.xml
660e320066ea47ae.jpg
660e320066ea47ae.xml
6652d4ccc348f012.jpg
6652d4ccc348f012.xml
66b1b2f9f64d7629.jpg
66b1b2f9f64d7629.xml
670cf53713ff7074.jpg
670cf53713ff7074.xml
68fb4d98cbcd1369.jpg
68fb4d98cbcd1369.xml
690c26d5132340d8.jpg
690c26d5132340d8.xml
6930c5140c3a50c3.jpg
6930c5140c3a50c3.xml
69f97a936ae504bb.jpg
69f97a936ae504bb.xml
6cd1c3564c6518cf.jpg
6cd1c3564c6518cf.xml
6d86acd7e2c6c207.jpg
6d86acd7e2c6c207.xml
6e339a4edabe2bbf.jpg
6e339a4edabe2bbf.xml
704087077f293745.jpg
704087077f293745.xml
70429e8ce159c4e5.jpg
70429e8ce159c4e5.xml
7220bd1de921ee51.jpg
7220bd1de921ee51.xml
756dc4aa7f9f110a.jpg
756dc4aa7f9f110a.xml
7607c938b6d5f873.jpg
7607c938b6d5f873.xml
7652eb4db0de809f.jpg
7652eb4db0de809f.xml
76a04d2de37572ce.jpg
76a04d2de37572ce.xml
77af20e6139ab1b4.jpg
77af20e6139ab1b4.xml
783eabdbd0b543eb.jpg
783eabdbd0b543eb.xml
78e1514ccd7fc0d1.jpg
78e1514ccd7fc

In [ ]:
def read_image_and_normalize(sample_path:str, resize_to=False) -> np.ndarray:
 
        
    im = Image.open(sample_path)
    if resize_to:
        im = im.resize(resize_to)
    im = np.array(im) / 255.0
    im = im.astype("float32")

    return im

In [ ]:
def load_data(train=True):
    array_images = []
    pbar = ProgressBar()
    if train:
        array_labels = []
        for idx, img in pbar(enumerate(df.image_id)):

            array_images.append(read_image_and_normalize(PATH_IMAGE+img))

        return np.array(array_images), np.array(array_labels)
    
        

# x_train, y_train = load_data(df_train, resize_to=(params['img_size'],params['img_size']))
# x_valid, y_valid = load_data(df_valid, resize_to=(params['img_size'],params['img_size']))

### Apply Data augmentation 
#### (Different data aug can be applied to test several models)

In [3]:
print("lorem ipsum")

lorem ipsum


### Split Data

In [4]:
print("lorem ipsum")

lorem ipsum


### Print Data and explain them (part of the project)

In [5]:
print("lorem ipsum")

lorem ipsum


### Load pretrain coco 2017 model (detect stop sign auto)

In [6]:
print("lorem ipsum")

lorem ipsum


### Train

In [7]:
print("lorem ipsum")

lorem ipsum


### Samples predicts

In [8]:
print("lorem ipsum")

lorem ipsum


### Analysis of result

In [9]:
print("lorem ipsum")

lorem ipsum


### Store Data for later (to compare results of data augs Tier 1, 2, 3, ...)

In [10]:
print("lorem ipsum")

lorem ipsum
